# Pre-processing Green's Functions calculated with SPECFEM3D for source-side MTUQ moment tensor estimations
### Felix Rodriguez-Cardozo and Jochen Braunmiller

#### We appreciate the colaboration of Liang Ding, University of Toronto, for double-checking these results and correct some bugs related to the scale factor for converting the GFs into the appropiate units required by MTUQ. 

-------------------------
 
[MTUQ has the capability of reading Green's Functions](https://uafgeotools.github.io/mtuq/user_guide/03.html) calculated in Specfem3D to incorporate 3D velocity models for moment tensor estimation.
 
This example aims to guide the pre-processing of the output plain-text files (synthetic seismograms) created by Specfem3D after a simulation. Then, convert them into SAC files with the appropriate name for being read by MTUQ as a Green's Functions database for a specific earthquake.
 
**Before starting save the information about the main directory location running the following cell**:

In [ ]:
import os
main_dir = os.getcwd()
print(main_dir)

In the same directory where you found this jupyter script, you will find the following directories:

In [ ]:
# Run this cell for seeing the basics directories for pre-processing the Specfem3D synthetics
%ls -d *

- **DATABASES_REPO**: contains the files for creating the mesh and the databases needed before running a synthetic seismograms simulation. 
- **SOLVER_REPO**: contains the files for running the synthetic seismograms simulations.
- **MTUQ_TEST**: contains the scripts for running a MTUQ moment tensor estimation example and testing that the pre-processed Green Functions work.
- **README.txt**: contains relevant information to read before working with this notebook.
- **make_specfem3D_GF.py**: python script for pre-processing Green's Functions. We will use it in the section 3 of this notebook. 


**Pre-requisites for using the preprocessing script**

This script works with the same python libraries installed in the mtuq anaconda environment. However, some parts of the script call Specfem3D binary files, and therefore, it is expected you have already installed  Specfem3D. Besides, the path to the binary files created during the Specfem3D installation has to be added to your bashrc profile. **If you have already calculated the synthetics for the elementary sources in Specfem3D, simply check that your files follow the directory structure shown here and go to the pre-processing part (section 3).** 

**Pre-processing overview**

The source-side workflow for MTUQ moment tensor estimation using Specfem3D Green Functions is summarized in the following figure.

In [ ]:
from IPython.display import Image

In [ ]:
Image('source_side_specfem3D_mtuq.png')

The following example, has been tested for running Specfem3D cartesian for a 1D velocity model, preprocessing the synthetic seismograms after Specfem3D simulations, and run an example in MTUQ.  After testing this notebook, you can use the same file structure and modify it according to your needs.


## 1. Creating the mesh and database in Specfem3D

The first part of the workflow consists of creating the mesh and the databases used by the Specfem3D. Internally, Specfem3D generates the mesh with the binary file **xmeshfem3D** and pours the velocity model into the mesh executing **xgenerate_databases** binary file. The directory **DATABASES_REPO** contains the files for creating the mesh and the databases for the given velocity model. Go to the directory **DATABASES_REPO/1D_VM/16_PROC** running the following cell:  

In [ ]:
%cd DATABASES_REPO/1D_VM/16_PROC
%ls *

**If you work  under Slurm workload manager, then run the following cell:**

In [ ]:
! sbatch schedule_databases.sh

**Otherwise, run the following cell:**

In [ ]:
#If you change the number of processes, then do it as well in the parameter files
! mpirun -np 16 xmeshfem3D
! mpirun -np 16 xgenerate_databases


Once you have created the mesh and generated the database, you will find the databases in **OUTPUT_FILES/DATABASES_MPI**, and you are ready to run the solver and simulate the synthetic seismograms. The dummy file before generating the databases is HI_DATABASES.txt.

In [ ]:
%ls OUTPUT_FILES/DATABASES_MPI

You do not need to run xmeshfem3D and xgenerate_databases again for different simulations as long as you use the same velocity model, mesh structure, and number of processes. For instance, the provided example is intended for running in 16 processors (that is why the path is 1D_VM/16_PROC). Future simulations with only modifications in the CMTSOLUTION file (source) and the STATIONS file (receivers’ location) can use the same database.

## 2. Running the solver

In [ ]:
#Let's back to the main directory
os.chdir(main_dir)

**2.1 Solver repository file structure**

Before launching the Specmfem3D solver, check the structure of the **SOLVER_REPO** directory: **SOLVER_REPO/event_id/depth_in_km**. The event id follows the format: **yyyymmddhhmmss**, where y stands for year, m for the month, d for day, and s for seconds. 

For this example you have already the structure for calculating the elementary sources for an earthquake on 2017/12/01 02:32:44 and 15 km of depth. Look at the directories running the following cell.

In [ ]:
%ls SOLVER_REPO/20171201023244/15/

You will see one directory per elementary moment tensor source: Mpp,Mrp,Mrr,Mrt,Mtp,Mtt. 
Each elementary source directory contains the directory structure for running the Specfem3D solver.

In [ ]:
#Let's see the content for the elementary source Mpp
%ls SOLVER_REPO/20171201023244/15/Mpp

The information regarding the source is in **DATA/CMTSOLUTION** file. Each elementary source **CMTSOLUTION** file  should contain the same information regarding the source hypocenter but differ in the moment tensor source content according to each elementary source. For instance the Mpp CMTSOLUTION file describe a source where all the moment tensor elements are zero except the Mpp one. See the **Mpp/DATA/CMTSOLUTION** file running the next cell.

In [ ]:
! cat SOLVER_REPO/20171201023244/15/Mpp/DATA/CMTSOLUTION

Now see see the **Mrp/DATA/CMTSOLUTION** and observe the difference in the source:

In [ ]:
! cat SOLVER_REPO/20171201023244/15/Mrp/DATA/CMTSOLUTION

**2.2 Launching the solver**

The script **make_specfem3D_GF.py** are in charge of running the Specfem3D solver, pre-processing the output files, and creating the Green Functions database to be used in MTUQ. Run the following cell for seeing the options:

In [ ]:
%run -i 'make_specfem3D_GF.py' -h

You have to decide, between:

- Run the solver for the six elementary moment tensor sources (option s). 
- Run pre-processing for the output of six elementary moment tensor sources (option  p).  

Besides, you have to provide the following information: 


- The event ID (-ev option)  in the format: yyyymmddhhmmss. y stands for year, m for the month, d for day, and s for seconds. 
- Earthquake depth in km (-ed option). 
- Path to database directory relative to the current script position.  




For launching the solver for all the elementary sources that come with the example provided, run the following cell:

In [ ]:
%run -i 'make_specfem3D_GF.py' -o s -ev 20171201023244 -ed 15 -dir DATABASES_REPO/1D_VM/16_PROC/

The script  have created soft symlink links between the **DATABASES_MPI** in the databases repository (indicated in the -dir option) and the **OUTPUT_FILES** directories for each elementary source in the solver repository (according to the information given in -ev and -ed options). See the directories’ symbolic links by running the following cell. 



In [ ]:
! ls -al SOLVER_REPO/20171201023244/15/M*/OUTPUT_FILES/DATABASES_MPI

Observe that all the **OUTPUT_FILES/DATABASES_MPI** directories created for the elementary sources are soft links pointing to the same **DATABASES_MPI** directory in the database repository.

Once created the symbolic links, the algorithm runs the shell **schedule_solver.sh**  for each elementary source. This instruction is printed but its execution is commented in the script (line 71)  because running the solver for each elementary source may take several hours. Besides, consider that **schedule_solver.sh** only works if you have the slurm workload manager (sbatch) installed in your system. Otherwise, once the algorithm has created the symbolic links, you must run the solver for each elementary source by typing in the terminal: **mpirun -np 16 xspecfem3D** at each **SOLVER_REPO/20171201023244/15/[Mpp,Mrp,Mrr,Mrt,Mtp,Mtt]** directory. 

## 3. Pre-processing Specfem3D output files

**3.1 Files required for pre-processing Specfem3D output files**

Once the synthetics are ready after running the solver for each elementary source, it is time to pre-process them and make a single directory with all the synthetics in the format required by MTUQ.  For the example provided, you can see the raw synthetic seismograms for the 6 elementary sources, running the following cell:


In [ ]:
! ls SOLVER_REPO/20171201023244/15/M*/OUTPUT_FILES/

For pre-processing all the synthetic seismograms, you need to provide information about the earthquake in a file named **eventinfo.txt**, and about the receivers in **STATIONS**. Both files are in SOLVER_REPO/20171201023244.


- **eventinfo.txt** follows the format YYYY-MM-DDTHH:MM:SS LATITUDE LONGITUDE DEPTH Mw.

Y stands for year, M  minute, D day, H hour, and S seconds. T is the letter that should be written between the event date and hour. 

In [ ]:
! cat SOLVER_REPO/20171201023244/eventinfo.txt

- **STATIONS** follows the same format than the Specfem3D stations file.

In [ ]:
! cat SOLVER_REPO/20171201023244/STATIONS

**3.2 Launch the pre-processing**

For launching the pre-processing use the option '-o p'. Run the following cell:

In [ ]:
%run -i 'make_specfem3D_GF.py' -o p -ev 20171201023244 -ed 15 -dir DATABASES_REPO/1D_VM/16_PROC/

**The pre-processing consists of:**

- Reading the raw synthetics seismograms for all the elementary sources and converting them into SAC files.

- Completing SAC files header information: event latitude, longitude, and depth; station latitude, longitude, and depth; network, and component.

- Integrating all the SAC files if they were converted from velocity synthetic seismograms (i.e., raw Specfem3D files end in .semv). if the Specfem3D simulations are done in displacement (i.e., raw Specfem3D files end in .semd), the pre-processing will skip the integration part.

- Scal amplitudes, consisting of normalizing by dividing all the synthetics seismograms by the source size defined in the CMTSOLLUTION file and then multiply them by 1e7 for converting from dyn.cm to N.m. Ideally, if the source size for calculating the GFs is just one, the scale process just consists on multiply all the synthetics for 1e7. For this example, the source for calculating the GFs has an amplitude of 1e23, and the script scales the synthetics dividing the signal by 1e16, that comes from diving the synthetics by  1e23 and then multyplying them by 1e7 (1e23/1e7 = 1e16).

- Rotating the sac files to the radial and transverse components.


If you succeed, you will see a new directory named PROCESSED in addition to the elementary source directories. The example provided created the directory in **SOLVER_REPO/20171201023244/15**.

The **PROCESSED** directory contains all the synthetic seismograms from the elementary sources in SAC format, preprocessed and named according to the MTUQ requirements for reading the green functions database. 

In [ ]:
os.chdir(main_dir)
! ls SOLVER_REPO/20171201023244/15/PROCESSED 

## 4. Run MTUQ using Specfem3D Green Functions

The last step is verifying that MTUQ can read the Green Functions database and run a moment tensor estimation. 

To verify the Green Functions calculated for the provided example, the directory **MTUQ_TEST** contains scripts for running a validation test. The following cell will take you to **MTUQ_TEST** and will show its content


In [ ]:
%cd MTUQ_TEST 
! ls

- **GFs**: Green Functions databases.
- **20171201023244**: observed data for the validation test. This 'observed data' are Specfem3D synthetics calculated with the same velocity model used for the Green Functions. 
- **EXPECTED_SOLUTIONS**: After running MTUQ, you can compare your results againts the ones in this directory. 
- **SPECFEM3D_GFs_GridSearch.DoubleCouple_SW_options.py**: script for running an MTUQ double couple grid search. 
- **SPECFEM3D_GFs_GridSearch.FMT_SW_options.py**: script for running an MTUQ full moment tensor grid search. 


For calculating the "observed data", we run a simulation in Specfem3D using the following source:

**Mrr:       1.0e+23
Mtt:       0.0
Mpp:      -1.0e+23
Mrt:       1.0e+23
Mrp:      -1.0e+23
Mtp:      -1.0e+23**

That corresponds to a pure deviatoric source (oblique thrust event with a CLVD percentage).

For seeing the input source, run the next cell.


In [ ]:
Image('Synthetic_input_source.png')

The directory **GFs** follows the pattern **GFs/id_event/depth/**.
For the given example, the Green Functions created in **SOLVER_REPO/20171201023244/15/PROCESSED/** have to be saved in  **MTUQ_TEST/GFs/20171201023244/15**. Run the following cell for creating the subdirectories in **20171201023244** and **15** in **GFs** and copy the Green Functions 

In [ ]:
%mkdir -p GFs/20171201023244/15 
%cp ../SOLVER_REPO/20171201023244/15/PROCESSED/* GFs/20171201023244/15/

**4.1 Double couple grid-search:**

The script **SPECFEM3D_GFs_GridSearch.DoubleCouple_SW_options.py** performs a double couple moment tensor estimation. 
Run the following cell for seing the options:

In [ ]:
%run -i 'SPECFEM3D_GFs_GridSearch.DoubleCouple_SW_options.py' -h 

The parameters shown in the next cell will launch the moment tensor estimation for the test event prepared for this example. 

In [ ]:
%run -i 'SPECFEM3D_GFs_GridSearch.DoubleCouple_SW_options.py' -event 20171201023244 -evla 30.734 -evlo 57.390 -evdp 15000 -mw 4.8 -time 2017-12-01T02:32:44.00000Z -np 50 -fb 10-100 

If you have an error like:

FileNotFoundError: [Errno 2] No such file or directory: 'tmp.png'

Then run the script directly in the terminal:

*mpirun -np 8 python SPECFEM3D_GFs_GridSearch.DoubleCouple_SW_options.py -event 20171201023244 -evla 30.734 -evlo 57.390 -evdp 15000 -mw 4.8 -time 2017-12-01T02:32:44.00000Z -np 50 -fb 10-100*

Open the file **20171201023244DC_waveforms.png** for seing the MTUQ output

In [ ]:
Image("20171201023244DC_waveforms.png")

The synthetics (red) are similar but not identical to the observed data (black). This is because the grid search is coarse and because the inversion is constrained for being double couple. However, the true source is deviatoric. As finer is the grid-search, which implies a larger number in the option **-np**, more similar the synthetics and observed seismograms will be. 

For seeing the misfit distribution associated to the grid-search, open the figure **20171201023244DC_misfit.png**.

In [ ]:
Image("20171201023244DC_misfit.png")

**4.1 Full moment tensor grid-search:**

The script **SPECFEM3D_GFs_GridSearch.FMT_SW_options.py** performs a full moment tensor estimation. 
Options are the same than the double couple script. Run the following cell for seeing the options:

In [ ]:
%run -i 'SPECFEM3D_GFs_GridSearch.FMT_SW_options.py' -h 

The following cell will launch a full moment tensor grid-search 

In [ ]:
%run -i 'SPECFEM3D_GFs_GridSearch.FMT_SW_options.py' -event 20171201023244 -evla 30.734 -evlo 57.390 -evdp 15000 -mw 4.8 -time 2017-12-01T02:32:44.00000Z -np 9 -fb 10-100

If you have an error like:

FileNotFoundError: [Errno 2] No such file or directory: 'tmp.png'

Then run the script directly in the terminal:

*mpirun -np 8 python SPECFEM3D_GFs_GridSearch.FMT_SW_options.py -event 20171201023244 -evla 30.734 -evlo 57.390 -evdp 15000 -mw 4.8 -time 2017-12-01T02:32:44.00000Z -np 9 -fb 10-100*

In [ ]:
Image("20171201023244FMT_cat_waveforms.png")

For seeing the misfit distribution on the lune plot associated to the grid-search, open the figure **20171201023244FMT_cat_misfit_mt.png**.

In [ ]:
Image("20171201023244FMT_cat_misfit_mt.png")

Observe that the synthetics and observed are almost identical and the retrieved source is almost the same than the input source. Eventually with a large enough number point per axis (**-np**), the retrieved source will be identical to the input one, and synthetics and "observed" data will be identical as well. You can compare your solutions against the ones saved in the directory EXPECTED_SOLUTIONS.